Generate Posterior distributions of Interventions on the Environment's (traffic signal control) Causal Graph

In [ ]:
import pandas as pd
import pymc as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('./data/data_trace_generation_sumo_desiredSpeed50_friction1.csv')
print(df.head)







<bound method NDFrame.head of       desiredSpeed  friction  speed  waitingTime  emergencyBraking  collisions
0        13.888889  1.000000  12.53     0.714448                 9          27
1        13.888889  0.921260  12.60     0.742518                17          35
2        13.888889  0.925472  12.46     0.751977                16          22
3        13.888889  0.965209  12.56     0.729994                10          25
4        13.888889  0.974237  12.42     0.875622                 5          29
...            ...       ...    ...          ...               ...         ...
1995      8.333333  0.924307   7.49     1.192152                 0          44
1996      8.333333  0.728548   7.47     0.972897                 2          38
1997      8.333333  0.701690   7.50     0.773050                 8          36
1998      8.333333  0.863973   7.37     1.465583                 0          39
1999      8.333333  0.805321   7.43     1.035610                 1          24

[2000 rows x 6 column

In [ ]:
#Example
# Generate some synthetic data
np.random.seed(123)
x = np.random.randn(100)
y = 2 * x + 1 + np.random.randn(100) * 0.5

# Define the Bayesian model
with pm.Model() as model:
    # Priors for the model parameters
    slope = pm.Normal('slope', mu=0, sigma=1)
    intercept = pm.Normal('intercept', mu=0, sigma=1)
    sigma = pm.HalfNormal('sigma', sigma=1)

    # Likelihood function
    likelihood = pm.Normal('y', mu=slope * x + intercept, sigma=sigma, observed=y)

    # Inference
    trace = pm.sample(2000, tune=1000)

# Summarize the posterior distribution
pm.summary(trace)

In [ ]:
#Fit total effect of Desired Speed or Friction
#shift in friction from 
#  
x = df['desiredSpeed']
#y = df['waitingTime'] #zero slope
y = df['collisions'] #0.2 slope

#x = df['friction']
#y = df['waitingTime'] #-7.9 slope
#y = df['collisions'] #-4.0 slope



with pm.Model() as model:
    #setup priors for the model parameters
    slope = pm.Normal('slope', mu=0, sigma=1)
    intercept = pm.HalfNormal('intercept', sigma=1)
    sigma = pm.HalfNormal('sigma', sigma=1)##???????

    # Likelihood function
    #likelihood = pm.Normal('y', mu= slope * x + intercept, sigma=sigma, observed=y)

    # Linear function for the mean
    mu = intercept + slope * x 
    
    # Ensure mu is positive
    mu_positive = pm.math.maximum(mu, 0) # or pm.Deterministic('mu_positive', pm.math.maximum(mu, 0)) for explicitly defining

    # Likelihood
    likelihood = pm.HalfNormal('y', sigma=mu_positive, observed=y) 


    # Inference
    trace = pm.sample(2000, tune=1000)

# Summarize the posterior distribution
pm.summary(trace)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [slope, intercept, sigma]


/opt/anaconda3/envs/pymc_env/lib/python3.12/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 39 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
slope,0.795,0.058,0.688,0.902,0.001,0.001,3391.0,4069.0,1.0
intercept,21.574,0.678,20.294,22.834,0.012,0.008,3343.0,4181.0,1.0
sigma,0.788,0.601,0.001,1.863,0.010,0.007,2428.0,1757.0,1.0
